In [1]:
using AMLET, RDST, Distributions, ForwardDiff

In [2]:
ngamma = 8

8

In [3]:
struct DummyBatch <: Batch
    IND::Array{LM_Individual}
    rng::MRG32k3a
end

import Base.iterate
function iterate(db::DummyBatch)
    state = 1
    reset_stream!(db.rng)
    if state <= length(db.IND)
        return MLM_Individual(db.IND[state], db.rng, ngamma), state+1
    else
        return nothing
    end
end
function iterate(db::DummyBatch, state::Int = 1)
    if state <= length(db.IND)
        next_substream!(db.rng)
        return MLM_Individual(db.IND[state], db.rng, ngamma), state+1
    else
        return nothing
    end
end

iterate (generic function with 271 methods)

In [4]:
gen = MRG32k3aGen([125,8765423,4546,6434,645,76465])
betaGen = next_stream(gen)
monteCarlosGen = next_stream(gen)

Full state of MRG32k3a generator:
Cg = [2459988903, 764778996, 4187205638, 4089700229, 517468743, 2169295582]
Bg = [2459988903, 764778996, 4187205638, 4089700229, 517468743, 2169295582]
Ig = [2459988903, 764778996, 4187205638, 4089700229, 517468743, 2169295582]

In [5]:
function beta(θ::Vector, rng)
    N = Normal.(θ[1:4], θ[5:8])
    return [rand(rng, N[1]), rand(rng, N[2]), rand(rng, N[3]), rand(rng, N[4])]
end

beta (generic function with 1 method)

In [6]:
gum = Gumbel()

Gumbel{Float64}(μ=0.0, θ=1.0)

In [7]:
function genIND(dists::Array{T, 1}, θ::Vector, m::Int) where T<: Distribution
    β = beta(θ, betaGen)
    N = length(dists)
    param = Array{Float64, 2}(undef, 0, N)
    for _ in 1:m
        param = [param ; (rand.(dists))']
    end
    best = argmax(param*β + rand(gum, m))
    return LM_Individual(param, best, 1)
end

genIND (generic function with 1 method)

In [8]:
θstar = [5,6,7,8, 1,1,1,1]

8-element Array{Int64,1}:
 5
 6
 7
 8
 1
 1
 1
 1

In [9]:
dists = [Normal(2, 4) for _ in 1:4]
genIND(dists, θstar, 5)

LM_Individual{Array{Float64,2}}([2.5409073118806416 4.178394435023822 -3.1299148421827887 1.645383576816343; -3.548557303341404 5.696298552067979 5.452040177475544 3.81385680817073; … ; 2.2957789489243625 -0.7435623718679816 -1.2017500310544644 6.141915084071501; 11.522456512414559 -5.2037241521257505 6.261968088582228 -1.3300736068221743], 2, 1)

In [10]:
db = DummyBatch([genIND(dists, θstar, 5) for _ in 1:1000], monteCarlosGen);

In [11]:
mlm = MLM(db);

In [12]:
#linear utilities of mixed logit model where each parameters of beta are assumed to follow an 
#independant exponnential Distribution beta

function EU(θ::Vector, X::Matrix, rng::Rigged{N}) where N
    β = beta(θ, rng)
    reset_stream!(rng)
    return X*β
end

function EU_i(θ::Vector, X::Matrix, i::Int64, rng::Rigged{N}) where N
    β = beta(θ, rng)
    reset_stream!(rng)
    return X[i, :]'*β
end

function ∇EU(θ::Vector, X::Matrix, rng::Rigged{N}) where N
    return vcat([∇EU_i(θ, X, i, rng) for i in 1:size(X, 1)]...)
end

function ∇EU_i(θ::Vector, X::Matrix, i::Int64, rng::Rigged{N}) where N
    function tmp(x::Vector)
        return EU_i(x, X, i, rng)
    end
    return ForwardDiff.gradient(tmp, θ)
end

uti = AMLET.LinearUtilitie(EU, ∇EU, EU_i, ∇EU_i)


AMLET.LinearUtilitie(EU, ∇EU, EU_i, ∇EU_i)

In [13]:
complete_Model!(mlm, uti, 20)

(::AMLET.var"#∇F!#17"{DummyBatch,AMLET.LinearUtilitie,Int64}) (generic function with 2 methods)

In [14]:
mlm.f(ones(8))

0.474484237438874

In [15]:
mlm.f(ones(8))

0.474484237438874

In [16]:
grad1 = zeros(8)
grad2 = zeros(8)

8-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [17]:
mlm.∇f!(ones(8), grad1)
mlm.∇f!(ones(8), grad2)

8-element Array{Float64,1}:
  0.023636934618073983
 -0.040791604335042364
 -0.08478103445877407 
 -0.15194320020119764 
  0.09050622696010474 
  0.08345486968423595 
  0.046977618512055355
  0.015775665922038315

In [18]:
grad1 == grad2

true

# Using the basic trust region (BTR) algorithm with BFGS approximation

In [19]:
using GERALDINE

In [20]:
 θstar

8-element Array{Int64,1}:
 5
 6
 7
 8
 1
 1
 1
 1

In [ ]:
x = OPTIM_BFGS(mlm.f, mlm.∇f!, x0 = 1.0*θstar, nmax = 200, verbose = true)

In [ ]:
grad = zeros(8)

In [ ]:
mlm.∇f!(x[1], grad)